In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [7]:
df=pd.read_csv('/Users/filiph/Desktop/Prague_zip.csv') #importing prague district ZIP codes
df

,District,ZIPcode
0,Praha 1,110 00
1,Praha 2,120 00
2,Praha 3,130 00
3,Praha 4,140 00
4,Praha 5,150 00
5,Praha 6,160 00
6,Praha 7,170 00
7,Praha 8,180 00
8,Praha 9,190 00
9,Praha 10,100 00


In [8]:
df.rename(columns={'ZIPcode':'Postal Code'}, inplace=True)
df.head()

,District,Postal Code
0,Praha 1,110 00
1,Praha 2,120 00
2,Praha 3,130 00
3,Praha 4,140 00
4,Praha 5,150 00


In [9]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="Prague_explorer")

df['Coordinates']= df["Postal Code"].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df[['Latitude', 'Longitude']] = df['Coordinates'].apply(pd.Series)

df.drop(['Coordinates'], axis=1, inplace=True)
df

,District,Postal Code,Latitude,Longitude
0,Praha 1,110 00,50.085415,14.425401
1,Praha 2,120 00,50.074726,14.433938
2,Praha 3,130 00,50.083118,14.451298
3,Praha 4,140 00,50.049204,14.440276
4,Praha 5,150 00,50.072087,14.395063
5,Praha 6,160 00,50.106810,14.393291
6,Praha 7,170 00,50.101154,14.432379
7,Praha 8,180 00,50.105209,14.461589
8,Praha 9,190 00,50.112918,14.495606
9,Praha 10,100 00,50.072773,14.494590


In [5]:
CLIENT_ID = 'OUCRJPLZH03WOZR2PNTLINOYZXTVWLSI42AT4CAVM5KCSOOY' #connecting to Foursquare API
CLIENT_SECRET = '52QU1HPMSOAAFTSCV4R4OYJ0WPVMXCBF2PLL0SOEVYV32RPF' 
VERSION = '20180604'
LIMIT = 50

In [10]:
address = 'Prague'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

50.0874654 14.4212535


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
prague_venues = getNearbyVenues(names=df['District'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Praha 1
Praha 2
Praha 3
Praha 4
Praha 5
Praha 6
Praha 7
Praha 8
Praha 9
Praha 10


In [11]:
print(prague_venues.shape) #getting the needed dataset with venue data as well as their particular neighbourhood
prague_venues.head()

(392, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Praha 1,50.085415,14.425401,Stavovské divadlo | The Estates Theater (Stavo...,50.085824,14.423380,Theater
1,Praha 1,50.085415,14.425401,Hamleys,50.085347,14.425668,Toy / Game Store
2,Praha 1,50.085415,14.425401,Xplore Fitness,50.085781,14.424609,Gym / Fitness Center
3,Praha 1,50.085415,14.425401,Brasileiro,50.086428,14.427451,Brazilian Restaurant
4,Praha 1,50.085415,14.425401,Tezenis,50.086139,14.426022,Lingerie Store


In [13]:
# one hot encoding - getting dummies for each venue category
prague_onehot = pd.get_dummies(prague_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
prague_onehot['Neighbourhood'] = prague_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [prague_onehot.columns[-1]] + list(prague_onehot.columns[:-1])
prague_onehot = prague_onehot[fixed_columns]

prague_onehot.head()

,Neighbourhood,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Praha 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Praha 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Praha 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Praha 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Praha 1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
prague_grouped = prague_onehot.groupby('Neighbourhood').mean().reset_index() # obtaining the frequency of each venue category in each neighbourhood
prague_grouped

,Neighbourhood,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,...,Used Bookstore,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Praha 1,0.00,0.02,0.020000,0.00,0.00,0.00,0.000000,0.00,0.020000,...,0.000000,0.020000,0.00,0.020,0.000000,0.02000,0.000000,0.000000,0.00,0.02
1,Praha 10,0.00,0.00,0.000000,0.00,0.00,0.00,0.026316,0.00,0.078947,...,0.000000,0.000000,0.00,0.000,0.026316,0.00000,0.026316,0.000000,0.00,0.00
2,Praha 2,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.02,0.000000,...,0.000000,0.060000,0.00,0.040,0.000000,0.00000,0.040000,0.000000,0.00,0.02
3,Praha 3,0.00,0.00,0.000000,0.00,0.00,0.02,0.000000,0.00,0.020000,...,0.000000,0.000000,0.00,0.000,0.000000,0.00000,0.020000,0.000000,0.00,0.00
4,Praha 4,0.02,0.00,0.000000,0.00,0.00,0.02,0.000000,0.00,0.060000,...,0.000000,0.060000,0.02,0.020,0.020000,0.00000,0.000000,0.000000,0.04,0.02
5,Praha 5,0.00,0.00,0.035714,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00,0.000,0.000000,0.00000,0.000000,0.000000,0.00,0.00
6,Praha 6,0.00,0.00,0.031250,0.00,0.00,0.00,0.000000,0.00,0.031250,...,0.000000,0.000000,0.00,0.000,0.000000,0.03125,0.031250,0.000000,0.00,0.00
7,Praha 7,0.00,0.00,0.000000,0.02,0.02,0.02,0.000000,0.00,0.020000,...,0.000000,0.040000,0.00,0.000,0.000000,0.00000,0.000000,0.020000,0.00,0.00
8,Praha 8,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.000000,0.000000,0.00,0.125,0.000000,0.00000,0.000000,0.000000,0.00,0.00
9,Praha 9,0.00,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.000000,...,0.027778,0.027778,0.00,0.000,0.000000,0.00000,0.000000,0.027778,0.00,0.00


In [15]:
num_top_venues = 10 # top venue categories in each neighbourhood

for hood in prague_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = prague_grouped[prague_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Praha 1----
                       venue  freq
0         Italian Restaurant  0.08
1                    Theater  0.06
2             Cosmetics Shop  0.04
3                       Café  0.04
4                      Hotel  0.04
5             Clothing Store  0.04
6           Toy / Game Store  0.02
7            Thai Restaurant  0.02
8  College Academic Building  0.02
9               Concert Hall  0.02


----Praha 10----
                venue  freq
0          Restaurant  0.08
1              Bakery  0.08
2   Food & Drink Shop  0.08
3         Pizza Place  0.05
4                Park  0.05
5        Burger Joint  0.05
6           Gastropub  0.05
7  Mexican Restaurant  0.03
8       Historic Site  0.03
9                Food  0.03


----Praha 2----
                           venue  freq
0                           Café  0.08
1                   Burger Joint  0.06
2  Vegetarian / Vegan Restaurant  0.06
3                            Bar  0.04
4                       Wine Bar  0.04
5          Vietnames

In [16]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [17]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = prague_grouped['Neighbourhood']

for ind in np.arange(prague_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(prague_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Praha 1,Italian Restaurant,Theater,Hotel,Cosmetics Shop,Clothing Store,Café,Jazz Club,Indie Movie Theater,Ice Cream Shop,Food Court
1,Praha 10,Restaurant,Bakery,Food & Drink Shop,Park,Pizza Place,Gastropub,Burger Joint,Skate Park,Bowling Alley,Drugstore
2,Praha 2,Café,Vegetarian / Vegan Restaurant,Burger Joint,Bar,Beer Bar,Wine Bar,Bistro,Vietnamese Restaurant,Escape Room,Italian Restaurant
3,Praha 3,Café,Bar,Cocktail Bar,Pub,Restaurant,Beer Store,Kebab Restaurant,Performing Arts Venue,Gym,Gym Pool
4,Praha 4,Clothing Store,Vegetarian / Vegan Restaurant,Bakery,Gym / Fitness Center,Cosmetics Shop,Coffee Shop,Café,Women's Store,Pet Store,Bistro


In [18]:
# set number of clusters
kclusters = 5

prague_grouped_clustering = prague_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(prague_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 3, 3, 1, 4, 3, 3, 2, 0], dtype=int32)

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

prague_merged = df


# merge prague_grouped with prague_data to add latitude/longitude for each neighborhood
prague_merged = prague_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='District')


prague_merged=prague_merged.dropna()
prague_merged["Cluster Labels"]=prague_merged["Cluster Labels"].astype(int)
prague_merged.head()

,District,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Praha 1,110 00,50.085415,14.425401,1,Italian Restaurant,Theater,Hotel,Cosmetics Shop,Clothing Store,Café,Jazz Club,Indie Movie Theater,Ice Cream Shop,Food Court
1,Praha 2,120 00,50.074726,14.433938,3,Café,Vegetarian / Vegan Restaurant,Burger Joint,Bar,Beer Bar,Wine Bar,Bistro,Vietnamese Restaurant,Escape Room,Italian Restaurant
2,Praha 3,130 00,50.083118,14.451298,3,Café,Bar,Cocktail Bar,Pub,Restaurant,Beer Store,Kebab Restaurant,Performing Arts Venue,Gym,Gym Pool
3,Praha 4,140 00,50.049204,14.440276,1,Clothing Store,Vegetarian / Vegan Restaurant,Bakery,Gym / Fitness Center,Cosmetics Shop,Coffee Shop,Café,Women's Store,Pet Store,Bistro
4,Praha 5,150 00,50.072087,14.395063,4,Pub,Grocery Store,Bistro,Park,Bus Stop,Gym,Laser Tag,Event Space,Electronics Store,Movie Theater


In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(prague_merged['Latitude'], prague_merged['Longitude'], prague_merged['District'], prague_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examining the clusters

In [24]:
prague_merged.loc[prague_merged['Cluster Labels'] == 0, prague_merged.columns[[1] + list(range(5, prague_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,190 00,Restaurant,Park,Playground,Czech Restaurant,Pharmacy,Electronics Store,Supermarket,Bus Stop,Dessert Shop,Lake
9,100 00,Restaurant,Bakery,Food & Drink Shop,Park,Pizza Place,Gastropub,Burger Joint,Skate Park,Bowling Alley,Drugstore


In [25]:
prague_merged.loc[prague_merged['Cluster Labels'] == 1, prague_merged.columns[[1] + list(range(5, prague_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,110 00,Italian Restaurant,Theater,Hotel,Cosmetics Shop,Clothing Store,Café,Jazz Club,Indie Movie Theater,Ice Cream Shop,Food Court
3,140 00,Clothing Store,Vegetarian / Vegan Restaurant,Bakery,Gym / Fitness Center,Cosmetics Shop,Coffee Shop,Café,Women's Store,Pet Store,Bistro


In [26]:
prague_merged.loc[prague_merged['Cluster Labels'] == 2, prague_merged.columns[[1] + list(range(5, prague_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,180 00,Playground,Restaurant,Boat or Ferry,Beer Garden,Tram Station,Golf Driving Range,Dance Studio,Vietnamese Restaurant,Czech Restaurant,Creperie


In [28]:
prague_merged.loc[prague_merged['Cluster Labels'] == 3, prague_merged.columns[[1] + list(range(5, prague_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,120 00,Café,Vegetarian / Vegan Restaurant,Burger Joint,Bar,Beer Bar,Wine Bar,Bistro,Vietnamese Restaurant,Escape Room,Italian Restaurant
2,130 00,Café,Bar,Cocktail Bar,Pub,Restaurant,Beer Store,Kebab Restaurant,Performing Arts Venue,Gym,Gym Pool
5,160 00,Café,Pub,Plaza,Gastropub,Park,Coffee Shop,Bus Stop,Bistro,Italian Restaurant,Food
6,170 00,Café,Bistro,Dessert Shop,Vegetarian / Vegan Restaurant,Coffee Shop,Pizza Place,Hotel,Farmers Market,Shopping Mall,Food & Drink Shop


In [29]:
prague_merged.loc[prague_merged['Cluster Labels'] == 4, prague_merged.columns[[1] + list(range(5, prague_merged.shape[1]))]]

,Postal Code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,150 00,Pub,Grocery Store,Bistro,Park,Bus Stop,Gym,Laser Tag,Event Space,Electronics Store,Movie Theater
